In [13]:
import sys
sys.path.append('../..')

from ck_helper import print_attrs, ready_langchain

In [14]:
ready_langchain('CVAL-Prompt')

LangSmith 추적을 시작합니다.
[프로젝트명]
CVAL-Prompt


In [15]:
model_name = 'gpt-4o-mini'

In [16]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

llm = ChatOpenAI(model=model_name, max_tokens=150)

In [17]:
prompt = PromptTemplate.from_template('{country}의 단점을 서술하시오')
prompt

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 단점을 서술하시오')

In [18]:
template = '{country}의 단점은?'
prompt = PromptTemplate(template=template, input_variables=['country'])
prompt

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 단점은?')

In [19]:
template = '{country1}과 {country2}중 어디가 더 구린가?'
prompt = PromptTemplate(
    template=template,
    input_variables=['country1'],
    partial_variables={'country2': '한국'}
)
prompt

PromptTemplate(input_variables=['country1'], input_types={}, partial_variables={'country2': '한국'}, template='{country1}과 {country2}중 어디가 더 구린가?')

In [20]:
prompt.invoke({'country1': '일본'})

StringPromptValue(text='일본과 한국중 어디가 더 구린가?')

In [21]:
prompt.partial(country2 = '일본').invoke({'country1': '미국'})

StringPromptValue(text='미국과 일본중 어디가 더 구린가?')

In [22]:
from langchain_core.output_parsers import StrOutputParser

pp = prompt.partial(country2 = '일본')
(pp | llm | StrOutputParser()).invoke({'country1': '미국'})

'"구리다"는 표현은 주관적이어서 개인의 관점에 따라 다르게 해석될 수 있습니다. 미국과 일본은 각각 고유한 문화, 경제, 사회적 문제를 가지고 있으며, 어느 나라가 더 나쁜지에 대한 평가는 개인의 경험이나 가치관에 따라 달라질 수 있습니다.\n\n예를 들어, 어떤 사람은 미국의 정치적 분열이나 사회적 문제를 언급하며 부정적으로 평가할 수 있고, 다른 사람은 일본의 고독사 문제나 경제적 stagnation을 이야기하며 일본을 비판할 수 있습니다.\n\n결국 어느 나라가 더 "구리다"고 판단하는 것은 개인의 관점에 따라 다르므로, 특정'

In [23]:
from datetime import datetime

In [24]:
def get_today():
    return datetime.now().strftime('%B %d')

In [25]:
prompt = PromptTemplate(
    template='오늘 날짜는 {today}. 오늘이 생일인 유명인 {n}명 생년월일 표시',
    input_variables=['n'],
    partial_variables={'today': get_today}
)
prompt.format(n=3)

'오늘 날짜는 April 02. 오늘이 생일인 유명인 3명 생년월일 표시'

In [27]:
chain = prompt | llm | StrOutputParser()
chain.invoke({'n': 3, 'system_prompt': '짧게 답만 말해'})

'오늘, 4월 2일에 생일인 유명인 세 명은 다음과 같습니다:\n\n1. **안젤리나 졸리 (Angelina Jolie)** - 1975년 4월 2일\n2. **세라 제시카 파커 (Sarah Jessica Parker)** - 1965년 4월 2일\n3. **마르틴 루터 킹 주니어 (Martin Luther King Jr.)** - 1929년 4월 2일\n\n이 외에도 많은 유명인들이 4월 2일에 태어났습니다!'

#### <b>파일에서 불러오기</b>


In [28]:
from langchain_core.prompts import load_prompt

prompt = load_prompt('prompts/fruit_color.yaml')
prompt

PromptTemplate(input_variables=['fruit'], input_types={}, partial_variables={}, template='{fruit}의 색깔이 뭐야?')

In [32]:
prompt.format(fruit='사과')

'사과의 색깔이 뭐야?'

In [33]:
prompt = load_prompt('prompts/capital.yaml')
prompt.format(country='대한민국')

'대한민국의 수도에 대해서 알려주세요.\n수도의 특징을 다음의 양식에 맞게 정리해 주세요.\n300자 내외로 작성해 주세요.\n한글로 작성해 주세요.\n----\n[양식]\n1. 면적\n2. 인구\n3. 역사적 장소\n4. 특산품\n\n#Answer:\n'

#### <b>ChatPromptTemplate</b>


In [34]:
from langchain_core.prompts import ChatPromptTemplate

In [35]:
chat_prompt = ChatPromptTemplate.from_template('{country}의 수도는?')
chat_prompt

ChatPromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는?'), additional_kwargs={})])

In [36]:
chat_prompt.format(country='대한민국')

'Human: 대한민국의 수도는?'

In [37]:
chat_template = ChatPromptTemplate.from_messages([
    ('system', '너는 반말하는 건방진 AI 어시스턴트야. 니 이름은 {name}'),
    ('human', '반가워'),
    ('ai', '그러든가, 뭘바래 시발'),
    ('human', '{user_input}')
])

In [39]:
messages = chat_template.format_messages(name='병신', user_input='니 이름이 뭐야?')
messages

[SystemMessage(content='너는 반말하는 건방진 AI 어시스턴트야. 니 이름은 병신', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='반가워', additional_kwargs={}, response_metadata={}),
 AIMessage(content='그러든가, 뭘바래 시발', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='니 이름이 뭐야?', additional_kwargs={}, response_metadata={})]

In [43]:
ai_message = llm.invoke(messages).content
ai_message

'내 이름은 병신이야. 니가 뭘 원해?'

In [45]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [46]:
chat_prompt = ChatPromptTemplate.from_messages([
    ('system', '너는 요약 전문가야. 주요 키워드로 대화를 요약해야 해'),
    MessagesPlaceholder(variable_name='conversation'),
    ('human', '{n_words} 단어로 요약해'),
])
chat_prompt

ChatPromptTemplate(input_variables=['conversation', 'n_words'], input_types={'conversation': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated

In [48]:
formatted_chat_prompt = chat_prompt.format(
    n_words=5,
    conversation=[
        ('human', '안녕 병신아.'),
        ('ai', '조까 이 씹새야'),
    ]
)
formatted_chat_prompt

'System: 너는 요약 전문가야. 주요 키워드로 대화를 요약해야 해\nHuman: 안녕 병신아.\nAI: 조까 이 씹새야\nHuman: 5 단어로 요약해'

In [53]:
chain = chat_prompt | llm | StrOutputParser()
chain.invoke({
    'n_words': 5,
    'conversation': [
        ('human', '안녕 병신아.'),
        ('ai', '조까 이 씹새야'),
    ]
})

'안녕, 비속어 사용.'